# Table of Contents
- [Creating a UGrid](#creating_a_ugrid)
- [Writing a UGrid out to a File](#writing_a_ugrid_out_to_a_file)
- [Reading a UGrid in from a File](#reading_a_ugrid_in_from_a_file)
- [Important Note on UGrid.locations](#important_note_on_ugrid_locations)
- [More on UGrid Cells](#more_on_ugrid_cells)
- [UGrid Operations](#ugrid_operations)

<a class='anchor' id='creating_a_ugrid'></a>
# Creating a UGrid
Below are the imports that we will be using to work with ugrids

In [1]:
from xms.grid.ugrid import UGrid
from xms.grid.ugrid import ugrid_utils

And here are some general imports we will be using

In [2]:
import time

<p>
Ugrids are the basic geometry that make up a grid.  They are made up of cells which are made up of points. <br>
Some important terminology concering UGrids are:<br>
    - Location: The actual x,y,z coordinates of a point/vertex in the UGrid.<br>
    - Point: The integer index of the point inside the list of locations (UGrid.locations)<br>
    - Cell: A group of locations that make up a shape.<br>
</p>

In [3]:
locations = [(5.0, 74.0, 0.0),
          (22.0, 74.0, 0.0),
          (23.63, 60.57, 0.0),
          (11.35, 55.51, 0.0),
          (2.86, 62.19, 0.0),
          (12.53, 66.17, 0.0)]

The code above creates a list of locations. Each location in the list is a triple
where each value in the triple corresponds to x,y,z coordinates of a point.  This ugrid is going
to be 2D, but we could make a 3D ugrid if we wanted to.

In [4]:
cellstream = [
    UGrid.cell_type_enum.TRIANGLE, 3, 0, 4, 5,
    UGrid.cell_type_enum.TRIANGLE, 3, 5, 4, 3,
    UGrid.cell_type_enum.TRIANGLE, 3, 5, 2, 1,
    UGrid.cell_type_enum.TRIANGLE, 3, 2, 5, 3,
    UGrid.cell_type_enum.TRIANGLE, 3, 5, 1, 0
]

The code above creates a list of cells.  Each five values in the list together represent a cell.

The first cell in the list is `UGrid.cell_type_enum.TRIANGLE, 3, 0, 4, 5`.

The first value that describes the cell is `UGrid.cell_type_enum.TRIANGLE`, which is the type of the cell.  The cell type determines how the points in a cell relate to each other and what shape the cell will take.

The next value, `3`, indicates how many points there are in the cell.  Because we put `3`, there should be three values that follow this value.

The next three values `0, 4, 5` are each indicies that tell the indexes in `locations` that hold the points that make up the cell.  The following code illustrates this.


In [5]:
index_point_1 = cellstream[2]
index_point_2 = cellstream[3]
index_point_3 = cellstream[4]
print(locations[index_point_1], locations[index_point_2], locations[index_point_3])

(5.0, 74.0, 0.0) (2.86, 62.19, 0.0) (12.53, 66.17, 0.0)


Now that we have created the locations and cells that will make up our UGrid, we can make a Ugrid by passing
the locations and cellstream lists to the UGrid constructor:

In [6]:
out_ugrid = UGrid(locations, cellstream)

There are many operations that can be performed on a UGrid.  Lets write out our UGrid to a file so we can import it into GMS and see what it looks like.

<a id='writing_a_ugrid_out_to_a_file'></a>

# Writing a UGrid out to a File

In [7]:
out_file_name = "output_files/test.xmugrid"
ugrid_utils.write_ugrid_to_ascii_file(out_ugrid, out_file_name)

When we take the .xmugrid file that we just created and import it into GMS we get this:

![The ugrid generated by our code imported into GMS](images/ugrid.png)

<a id="reading_a_ugrid_in_from_a_file"></a>
# Reading a UGrid in from a File

Just as we wrote out the UGrid to a file, we can read in a UGrid from a file.

In [8]:
in_ugrid = ugrid_utils.read_ugrid_from_ascii_file(out_file_name)
print(f'out_ugrid: {out_ugrid}')
print(f'in_ugrid: {in_ugrid}')

out_ugrid: <UGrid - Number of Locations: 6, Number of Cells: 5, Extents: ((2.86, 55.51, 0.0), (23.63, 74.0, 0.0)), Modified: True>
in_ugrid: <UGrid - Number of Locations: 6, Number of Cells: 5, Extents: ((2.86, 55.51, 0.0), (23.63, 74.0, 0.0)), Modified: True>


<a id="important_note_on_ugrid_locations"></a>
# Important Note on UGrid.locations
The UGrid `in_ugrid` that we read in has the same data as the UGrid `out_ugrid` that we created.
We can get at that data, but there is an important note on getting UGrid.locations.  Each time that
you call the UGrid.locations property, a new array is created and returned with a copy of the locations data.

In [9]:
in_ugrid.locations

array([[ 5.  , 74.  ,  0.  ],
       [22.  , 74.  ,  0.  ],
       [23.63, 60.57,  0.  ],
       [11.35, 55.51,  0.  ],
       [ 2.86, 62.19,  0.  ],
       [12.53, 66.17,  0.  ]])

When there are a lot of cells in the UGrid this can be expensive.  If we call to it multiple times watch how much time it takes.

In [10]:
def get_time():
    return time.perf_counter_ns()

_iterations = 100000

In [11]:
# Calling UGrid.locations each time
time_start = get_time()

for i in range(_iterations):
    _locations = in_ugrid.locations
    for cell in _locations:
        cell

time_end = get_time()

time_calling_each_time = time_end - time_start

In [12]:
# Calling UGrid.locations once
time_start = get_time()

_locations = in_ugrid.locations

for i in range(_iterations):
    for cell in _locations:
        cell

time_end = get_time()

time_calling_once = time_end - time_start

In [13]:
print(f'Calling UGrid.locations each time: {time_calling_each_time}')
print(f'Calling UGrid.locations once: {time_calling_once}')
print(f'Difference (calling_each_time - calling_once): {time_calling_each_time - time_calling_once}')
print(f'Calling UGrid.locations each time took {time_calling_each_time / time_calling_once} times longer in this scenario')

Calling UGrid.locations each time: 238844300
Calling UGrid.locations once: 121057200
Difference (calling_each_time - calling_once): 117787100
Calling UGrid.locations each time took 1.9729871498762568 times longer in this scenario


It is important to not make more calls to UGrid.locations than necessary as this can be a slow process.

<a id="more_on_ugrid_cells"></a>
# More on Ugrid Cells
Remember that a ugrid cell is defined by a sequence of numbers, called a cell stream. A cell stream is composed of the following parts:
 - First, a numerical declaration of the shape
 - Second, the number of points
 - Third, the point indices
 
There are many different kinds of cells available, each specific to 2D or 3D shapes.
- Supported 2D grid cells include: triangle (5), polygon (7), pixel (8), quad (9).
- Supported 3D grid cells include: tetrahedron (10), voxel (11), hexahedron (12), wedge (13), pyramid (14), and polyhedron (42).

An example cell stream:
```
|- Shape declaration
|  |- # of points
|  |  |- Indicies
|  |  |-----------|
7, 5, 0, 1, 2, 3, 4
```

The cell definitions mirror VTK cell definitions which are available on page 9 of VTK File Formats for VTK version 4.2 at https://www.vtk.org/wp-content/uploads/2015/04/file-formats.pdf.

For convinience, the `xmugrid_celltype_enum` was created to put human-readable names to the numerical shape declarations.

### Detailed Examples of Cell Streams

![Each kind of cell shape](images/Shapes.png)

#### 2D
A triangle (5) has 3 points and the points are declared in a counter-clockwise direction. A cell stream example for a triangle is: `5, 3, 0, 1, 2`.

A polygon (7) does not a have a defined number of points, but the points are still declared in a counter-clockwise direction. A cell stream example for a polygon is: `7, 5, 0, 1, 2, 3, 4`.

A pixel (8) has 4 orthogonially defined points. A cell stream example for a pixel is: `8, 4, 0, 1, 2, 3`.

A quad (9) has 4 points declared in a counter-clockwise direction. A cell stream example for a quad is: `9, 4, 0, 1, 2, 3`.

#### 3D
A tetrahedron (10) has 4 points. A cell stream example for a tetrahedron is: `10, 4, 0, 1, 2, 3`.

A voxel (11) has 8 orthogonially defined points. A cell stream example for a voxel is: `11, 8, 0, 1, 2, 3, 4, 5, 6, 7`.

A hexahedron (12) has 8 points. A cell stream example for a hexahedron is: `12, 8, 0, 1, 2, 3, 4, 5, 6, 7`.

A wedge (13) has 6 points. A cell stream example for a wedge is: `13, 6, 0, 1, 2, 3, 4, 5`.

A pyramid (14) has 5 points. A cell stream example for a pyramid is: `14, 5, 0, 1, 2, 3, 4`.

A polyhedron (42) does not a have a defined number of points. A polyhedron cell stream has the following format: The cell type, number of faces, and then repeated for each face the number of points in the face followed by the points in the face declared in a counter-clockwise direction. A cell stream example for a polyhedron is: `42, 6, 4, 10, 13, 12, 11, 4, 10, 11, 15, 16, 4, 11, 12, 16, 15, 4, 12, 13, 17, 16, 4, 13, 10, 14, 17, 4, 14, 15, 16, 17`.

Below is an example of a UGrid that has a combination of different cell shapes

In [14]:
pts = [
    (0, 0, 0), (2, 0, 0), (1, 2, 0), (4, 1, 0), (3, 3, 0), (-1, 3, 0), (-2, 1, 0),
    (0, -2, 0), (2, -2, 0)
]

cells = [
    UGrid.cell_type_enum.TRIANGLE, 3, 0, 1, 2,
    UGrid.cell_type_enum.QUAD, 4, 1, 3, 4, 2,
    UGrid.cell_type_enum.QUAD, 4, 2, 5, 6, 0,
    UGrid.cell_type_enum.QUAD, 4, 0, 7, 8, 1,
    UGrid.cell_type_enum.TRIANGLE, 3, 1, 8, 3,
    UGrid.cell_type_enum.TRIANGLE, 3, 2, 4, 5,
    UGrid.cell_type_enum.TRIANGLE, 3, 0, 6, 7,
]

ugrid_multiple_cell_types = UGrid(pts,cells)

![A UGrid made up of various cell types](images/ugrid_multiple_cell_types.png)

<a id="ugrid_operations"></a>
# UGrid Operations

Lets take a look at some of the operations available to UGrids.  First we'll look at operations related to locations/points.

#### UGrid.point_count
We can get the number of points with the property UGrid.point_count:

In [15]:
in_ugrid.point_count

6

#### UGrid.get_point_location(index)
We can get the actual x,y,z coordinates of a point (the point location) with the `UGrid.get_point_location(index)` method. It takes an index and returns a Tuple.

In [16]:
in_ugrid.get_point_location(0)

(5.0, 74.0, 0.0)

The index is 0 based and goes up to `UGrid.point_count - 1`. If you pass it an index outside that range it returns a Tuple filled with zeros:

In [17]:
print(in_ugrid.get_point_location(-1))
in_ugrid.get_point_location(6)

(0.0, 0.0, 0.0)


(0.0, 0.0, 0.0)

#### UGrid.set_point_location(index, location)
We can set the location of a point with the `UGrid.set_point_location(index, location)` method. It takes an index, a Tuple, and returns a Boolean indicating whether the method was successful or not.

In [18]:
result = in_ugrid.set_point_location(0,(8.0,74.0,0.0))
result

True

In [19]:
result = in_ugrid.set_point_location(-1,(0,0,0))
result

False

#### Other Methods
Here are two methods that are similar to the previous two methods:

In [20]:
_r1 = in_ugrid.get_point_xy0(0)              # The same as get_point_location(), except the z coordinate is set to 0 in the returned Tuple
_r2 = in_ugrid.get_points_locations((0,1))   # The same as get_point_location(), except it can be passed multiple indices and returns a list of results

print(f'get_point_xy0(0): {_r1}')
print(f'get_points_locations((0,1)): {_r2}')

get_point_xy0(0): (8.0, 74.0, 0.0)
get_points_locations((0,1)): [[ 8. 74.  0.]
 [22. 74.  0.]]


The extents of a grid are the minimum and maximum points on the grid for each dimension. It returns two tuples both of which hold three values. The first tuple holds the minimum for each dimension (x,y,z), and the second tuple holds the maximum for each dimension (x,y,z)

In [21]:
# get_extents()

extents_min, extents_max = in_ugrid.extents

print(f"Extents Minimum: {extents_min}")
print(f"Extents Maximum: {extents_max}")

Extents Minimum: (2.86, 55.51, 0.0)
Extents Maximum: (23.63, 74.0, 0.0)


`get_point_adjacent_cells(point)` returns the cells that are adjacent to the given point (remember that a point is an index of a location).

In [22]:
# get_point_adjacent_cells()

cell_indices = in_ugrid.get_point_adjacent_cells(2)

print(f"Adjacent Cells: {cell_indices}")

Adjacent Cells: (2, 3)


`get_points_adjacent_cells(points)` is the same as get_point_adjacent_cells(), except it takes multiple points.

In [23]:
# get_points_adjacent_cells()

point_indices = [0, 1]
cell_indices = in_ugrid.get_points_adjacent_cells(point_indices)

print(f"Adjacent Cells: {cell_indices}")

Adjacent Cells: (4,)


## Cell Methods and Properties
Here are some methods and properties releated to cells:<br>

#### UGrid.cell_count
We can get the number of cells in the UGrid with UGrid.cell_count:

In [24]:
in_ugrid.cell_count

5

#### UGrid.get_cell_points(index)
Gets the points (indices) for the locations in UGrid.locations that make up the cell at the given index. Each entry in the return value is an index.

In [25]:
in_ugrid.get_cell_points(0)   # Gets the points of a cell (including polyhedron)

(0, 4, 5)

#### UGrid.get_cell_locations(index)
Gets the actual x,y,z coordinates of the points that make up the cell at the given index

In [26]:
in_ugrid.get_cell_locations(0)

array([[ 8.  , 74.  ,  0.  ],
       [ 2.86, 62.19,  0.  ],
       [12.53, 66.17,  0.  ]])

#### UGrid.get_cell_dimension(index) 
Gets the number of dimensions of the cell at the given index

In [27]:
print(f'{in_ugrid.get_cell_dimension(0)}')
print(f'{in_ugrid.get_cell_dimension(-1)}')   # Bad input returns -1

2
-1


#### UGrid.get_dimension_counts()
Gets the number of cells that belong to each dimension

In [28]:
# get_dimension_counts()

_0d, _1d, _2d, _3d = in_ugrid.dimension_counts

print(f"Number of 0d dimensions: {_0d}")
print(f"Number of 1d dimensions: {_1d}")
print(f"Number of 2d dimensions: {_2d}")
print(f"Number of 3d dimensions: {_3d}")

Number of 0d dimensions: 0
Number of 1d dimensions: 0
Number of 2d dimensions: 5
Number of 3d dimensions: 0


#### UGrid.get_cell_type(index)
Gets the type of the cell with the given index

In [29]:
# get_cell_type()

cell_type_0 = ugrid_multiple_cell_types.get_cell_type(0)
cell_type_1 = ugrid_multiple_cell_types.get_cell_type(2)

print(f"Cell 0 Type: {cell_type_0}")
print(f"Cell 1 Type: {cell_type_1}")

Cell 0 Type: ugrid_celltype_enum.TRIANGLE
Cell 1 Type: ugrid_celltype_enum.QUAD


#### get_cell_cellstream()
Returns a tuple holding whether the operation was succesful or not, and the cellstream requested (if successful).<br>
The cellstream consists of a tuple that holds the type of cell, the number of points that make up the cell, and the points of the cell.

In [30]:
# get_cell_cellstream()

successful_2, cellstream_2 = in_ugrid.get_cell_cellstream(2)
successful_3, cellstream_3 = in_ugrid.get_cell_cellstream(3)
successful_99, _ = in_ugrid.get_cell_cellstream(99)

if successful_2:
    print(f"Cellstream for cell 2: {cellstream_2}")
if successful_3:
    print(f"Cellstream for cell 3: {cellstream_3}")
if not successful_99:
    print(f"Cellstream 99 failed as expected...")

Cellstream for cell 2: (5, 3, 5, 2, 1)
Cellstream for cell 3: (5, 3, 2, 5, 3)
Cellstream 99 failed as expected...


#### get_cell_adjacent_cells(index)
Returns the cells that are adjacent to the cell with the given index

In [31]:
# get_cell_adjacent_cells()

adjacent_cells = in_ugrid.get_cell_adjacent_cells(3)

print(f"Adjacent cells to cell 3: {adjacent_cells}")

Adjacent cells to cell 3: (2, 0, 1, 4)


## Cell Edges
You can simply iterate over the edges of a cell with the following code:

In [32]:
edges = in_ugrid.get_cell_edges(0)

for edge in edges:
    print(edge)

(0, 4)
(4, 5)
(5, 0)


Each edge tuple holds the indices of the locations in UGrid.locations that make up the edge. <br>
You can get the points of an edge with `get_cell_edge(cell_index, edge_index)`. You pass it the index of the cell, and the index of the edge in that cell of which you want the points of. The tuple returned holds the points that make up the specified edge.

In [33]:
#### get_cell_edge()

edge_start, edge_end = in_ugrid.get_cell_edge(4, 2)

print(f"Edge Start Index: {edge_start}")
print(f"Edge End Index: {edge_end}")

Edge Start Index: 0
Edge End Index: 5


You can get the number of edges that make up a cell with get_cell_edge_count(index)<br>

In [34]:
# get_cell_edge_count()

cell_edge_count_0 = in_ugrid.get_cell_edge_count(0)
cell_edge_count_1 = in_ugrid.get_cell_edge_count(1)

print(f"Cell edge count for cell 0: {cell_edge_count_0}")
print(f"Cell edge count for cell 1: {cell_edge_count_1}")

Cell edge count for cell 0: 3
Cell edge count for cell 1: 3


There are useful methods that can tell you information about neighboring cells, such as `UGrid.get_edge_adjacent_cells(edge)`

In [35]:
_r1 = in_ugrid.get_edge_adjacent_cells((0, 4))
_r2 = in_ugrid.get_edge_adjacent_cells((4, 5))
print(_r1)
print(_r2)

(0,)
(0, 1)


This will return a tuple of indexes of the cells that are adjacent to the given cells. <br>

There are too many functions and properties of UGrids to cover here, but the ones covered should give you an idea of what kinds of operations exist.